In [1]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

from backtesting import Strategy
from backtesting import Backtest

/opt/homebrew/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Read in SPY data from last 18 years
dfSPY=yf.download("SPY",start='2005-01-01', end='2023-07-31')
dfSPY.head()

# Calculate Simple Moving Average and Relative Strength Index and add to dataframe
dfSPY['SMA']=ta.sma(dfSPY.Close, length=200)
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=4)

# Add Bollinger Band values (upper/lower) to the dataframe
my_bbands = ta.bbands(dfSPY.Close, length=20, std=2.5)
dfSPY=dfSPY.join(my_bbands)

# Remove rows with no data on 200 day moving average
dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [3]:
# Function to add SMA trend signal
def addsmasignal(df, backcandles):
    smasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.SMA[i]:
                dnt=0
            if df.Low[i]<=df.SMA[i]:
                upt=0
        if upt==1 and dnt==1:
            smasignal[row]=3
        elif upt==1:
            smasignal[row]=2
        elif dnt==1:
            smasignal[row]=1
    df['SMASignal'] = smasignal

addsmasignal(dfSPY, 5)

In [4]:
# Add orders to dataframe
def addorderslimit(df):
    ordersignal=[0]*len(df)
    for i in range(0, len(df)):
        if (df.SMASignal[i]==2 and df.Close[i]<=df['BBL_20_2.5'][i]) or (df.SMASignal[i]==1 and df.Close[i]>=df['BBU_20_2.5'][i]):
            ordersignal[i]=df.Close[i]
    df['ordersignal']=ordersignal
    
addorderslimit(dfSPY)

dfSPY[dfSPY['ordersignal'] > 0]

,Date,Open,High,Low,Close,Adj Close,Volume,SMA,RSI,BBL_20_2.5,BBM_20_2.5,BBU_20_2.5,BBB_20_2.5,BBP_20_2.5,SMASignal,ordersignal
146,2006-05-17,128.669998,129.100006,126.769997,126.849998,90.453651,144789500,125.864500,9.524091,127.482382,130.819500,134.156617,5.101865,-0.094750,2,126.849998
147,2006-05-18,127.349998,127.750000,126.110001,126.209999,89.997284,87906300,125.873600,7.963149,126.406025,130.573499,134.740973,6.383338,-0.023519,2,126.209999
341,2007-02-27,143.880005,144.199997,139.000000,139.500000,100.912277,274466500,134.407600,4.773477,141.000182,144.649000,148.297817,5.045064,-0.205571,2,139.500000
445,2007-07-26,150.190002,150.800003,146.389999,148.020004,107.963531,467592500,144.823750,12.647359,148.037739,152.643501,157.249262,6.034664,-0.001925,2,148.020004
446,2007-07-27,148.210007,148.869995,145.050003,145.110001,105.841026,422987600,144.873850,7.947565,146.304350,152.380000,158.455650,7.974341,-0.098290,2,145.110001
891,2009-05-04,88.550003,90.940002,88.379997,90.879997,69.056793,287120000,96.321000,89.030131,80.607974,85.632500,90.657025,11.735091,1.022188,1,90.879997
1073,2010-01-22,111.199997,111.739998,109.089996,109.209999,84.302834,345942400,100.878450,15.494690,109.716015,113.227000,116.737985,6.201675,-0.072062,2,109.209999
1362,2011-03-16,128.149994,128.570007,125.279999,126.180000,99.314644,468670300,118.417500,14.069454,126.747217,131.597000,136.446782,7.370658,-0.058479,2,126.180000
1418,2011-06-06,130.089996,130.360001,128.869995,129.039993,102.006920,179951200,125.088850,16.796619,129.077252,133.201498,137.325744,6.192492,-0.004517,2,129.039993
1631,2012-04-10,137.949997,138.339996,135.759995,135.899994,109.687813,235360300,127.208100,10.119874,136.965858,140.124999,143.284140,4.509032,-0.168695,2,135.899994


# Visualization

In [5]:
def pointposbreak(x):
    if x['ordersignal']!=0:
        return x['ordersignal']
    else:
        return np.nan
dfSPY['pointposbreak'] = dfSPY.apply(lambda row: pointposbreak(row), axis=1)

dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [6]:
# Backtesting
class MyStrat(Strategy):
    initsize = 0.99
    ordertime=[]
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        
        for j in range(0, len(self.orders)):
            if self.data.index[-1]-self.ordertime[0]>5:
                self.orders[0].cancel()
                self.ordertime.pop(0)   
            
        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=10:
                self.trades[-1].close()
            
            if self.trades[-1].is_long and self.data.RSI[-1]>=50:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=50:
                self.trades[-1].close()
        
        if self.signal!=0 and len(self.trades)==0 and self.data.SMASignal==2:
            # Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            # Add replacement order
            self.buy(sl=self.signal/2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])
        
        elif self.signal!=0 and len(self.trades)==0 and self.data.SMASignal==1:
            # Cancel previous orders
            for j in range(0, len(self.orders)):
                self.orders[0].cancel()
                self.ordertime.pop(0)
            # Add replacement order
            self.sell(sl=self.signal*2, limit=self.signal, size=self.initsize)
            self.ordertime.append(self.data.index[-1])

bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/10, commission=.00)
stat = bt.run()
stat

/var/folders/3p/flj5yn3j613fqtwq35mvt42m0000gn/T/ipykernel_46366/2017776604.py:44: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(dfpl, MyStrat, cash=10000, margin=1/10, commission=.00)


Start                                     0.0
End                                    4474.0
Duration                               4474.0
Exposure Time [%]                    4.178771
Equity Final [$]                271045.369255
Equity Peak [$]                 299820.938042
Return [%]                        2610.453693
Buy & Hold Return [%]              283.611797
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -53.082842
Avg. Drawdown [%]                  -14.103681
Max. Drawdown Duration                  896.0
Avg. Drawdown Duration              77.057143
# Trades                                 31.0
Win Rate [%]                        83.870968
Best Trade [%]                       4.474111
Worst Trade [%]                     -2.453217
Avg. Trade [%]                    

In [7]:
bt.plot(show_legend=False)

GridPlot(id='p1306', ...)